# Model for color detection

## Import libraries

In [1]:
import numpy as np
import pandas as pd
import math
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, regularizers
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, ConfusionMatrixDisplay, recall_score, classification_report

#from google.colab import drive
#drive.mount('/content/drive')

seed = 157 # fix randomisation
np.random.seed(seed)
tf.random.set_seed(seed)

2023-04-18 08:45:29.283814: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
pip install scikeras

In [ ]:
from scikeras.wrappers import KerasClassifier

## Import dataset

In [ ]:
# color_set = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/UCL/Assessment/color_dataset.csv')
color_set = pd.read_csv('/Users/mariebourel/Documents/Fac/Master 2022_2023/UCL/cours/CASA0018 - Deep Learning for sensor network /Coursework/color_dataset.csv')
color_set.head()

## Data analysis


In [ ]:
color_set.info()

In [ ]:
color_set.head()

In [ ]:
color_set.columns

In [ ]:
nb_col=len(color_set.columns)

## Data Preprocessing


### Data Encoding
Converting the label into binary variables using One-Hot Encoding as there is no order in the label

In [ ]:
df_colors = pd.get_dummies(color_set, columns=['label'])
df_colors.head()

### Data Normalization
Transform the RGB value from 0 to 255 to values from 0 to 1

In [ ]:
df_colors['red'] = df_colors['red'] / 255.0
df_colors['green'] = df_colors['green'] / 255.0
df_colors['blue'] = df_colors['blue'] / 255.0

In [ ]:
df_colors.head()

### Function to summarise

In [ ]:
def data_prep(df_data):
    # normalization input
    df_data['red'] = df_data['red'] / 255.0
    df_data['green'] = df_data['green'] / 255.0
    df_data['blue'] = df_data['blue'] / 255.0
    
    return df_data

### Split the dataset (70/15/15)

Splitting the dataset into 2 sets:
  - 85% for the training set (used to get the parameters of the model) including 15% for validation set (used to define the hyperparameter and avoid overfitting)
  - 15% for test set (used to test the model on unseen data)

In [ ]:
# sample the dataset
df = shuffle(df_colors)

# split between xs an ys
xs, ys = np.split(df.values, [3], axis=1)
#ys = ys.reshape(-1)

In [ ]:
xs_train, xs_test, ys_train, ys_test = train_test_split(xs, ys, test_size = 0.2, random_state = seed)

## Modelling

### ANN

#### Model 1

Define and compile the model

In [ ]:
# Parameters model 1
nb_layers1 = 2
activation_fct = 'relu'
loss_function = 'mean_squared_error'
optimizer = 'adam'
xs_shape = len(pd.DataFrame(xs_train).keys())

# model 1
model1 = keras.Sequential([
    layers.Dense(3, input_shape=[xs_shape], activation=activation_fct),
    layers.Dense(16, activation = activation_fct),
    layers.Dense(11)
])
model1.summary()

# compile the model 1
model1.compile(optimizer=optimizer, loss=loss_function, metrics=['accuracy'])

Train the model 1

In [ ]:
# fit the model with the Train set
training_history1 = model1.fit(x=xs_train, y=ys_train, 
                               validation_split=0.15, epochs=200,
                               batch_size=100)

# best accuracy 0.4671

In [ ]:
hist1 = pd.DataFrame(training_history1.history)
hist1['epoch'] = training_history1.epoch
hist1.tail()

In [ ]:
# Accuracy on training sets
accuracy_m1_train = model1.evaluate(xs_train, ys_train) # loss: 0.0329 - accuracy: 0.7606

print('Accuracy on train set', accuracy_m1_train)

In [ ]:
plt.plot(hist1['accuracy'])
plt.plot(hist1['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

Evaluate and predict on the test set

In [ ]:
model1_test_predict = model1.evaluate(xs_test, ys_test) # loss: 0.0294 - accuracy: 0.7814

In [ ]:
test_prediction1 = model1.predict(xs_test)
print("shape is {}".format(test_prediction1.shape))  
test_prediction1

In [ ]:
color_class1 = model1.predict(xs_test)

print(color_class1[8])
print(ys_test[8]) # color 2 ie blue

#### Model 2

Define and compile the model

In [ ]:
# Parameters model 2
activation_fct = 'relu'
loss_function = 'mean_squared_error'
optimizer = 'adam'

# model 2
model2 = keras.Sequential([
    layers.Dense(3, kernel_regularizer=regularizers.l2(0.001), activation=activation_fct, input_shape=[3]),
    layers.Dense(24, kernel_regularizer=regularizers.l2(0.001), activation=activation_fct),
    layers.Dense(24, kernel_regularizer=regularizers.l2(0.001), activation=activation_fct),
    layers.Dense(16, kernel_regularizer=regularizers.l2(0.001), activation=activation_fct),
    layers.Dense(11)
  ])
model2.summary()

# compile the model 2
model2.compile(optimizer=optimizer, loss=loss_function, metrics=['accuracy'])


Train the model 2

In [ ]:
training_history2 = model2.fit(x=xs_train, y=ys_train, 
                               validation_split=0.15, epochs=200, 
                               batch_size=100, verbose=1,
                               #callbacks=[tfdocs.modeling.EpochDots()], 
                               shuffle=True)

In [ ]:
hist2 = pd.DataFrame(training_history2.history)
hist2['epoch'] = training_history2.epoch
hist2.tail()

In [ ]:
# Accuracy on training sets
accuracy_m2_train = model2.evaluate(xs_train, ys_train) # loss: 0.0420 - accuracy: 0.7585

print('Accuracy on train set', accuracy_m2_train)

In [ ]:
plt.plot(hist2['accuracy'])
plt.plot(hist2['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

Predict on the test set

In [ ]:
model2_test_predict = model2.evaluate(xs_test, ys_test) #  loss: 0.0430 - accuracy: 0.7349

In [ ]:
test_prediction2 = model2.predict(xs_test)
print("shape is {}".format(test_prediction2.shape))  
test_prediction2

#### Model 3

Define and compile the model

In [ ]:
# Parameters model 3
activation_fct = 'relu'
loss_function = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
optimizer = keras.optimizers.Adam(learning_rate=0.001)

# model 3
model3 = keras.Sequential([
    layers.Dense(3, kernel_regularizer=regularizers.l2(0.001), activation=activation_fct, input_shape=(3,)),
    layers.Dense(24, kernel_regularizer=regularizers.l2(0.001), activation=activation_fct),
    layers.Dense(24, kernel_regularizer=regularizers.l2(0.001), activation=activation_fct),
    layers.Dense(16, kernel_regularizer=regularizers.l2(0.001), activation=activation_fct),
    layers.Dense(11)
  ])
model3.summary()

# compile the model 3
model3.compile(optimizer=optimizer, loss=loss_function, metrics=['accuracy'])


In [ ]:
training_history3 = model3.fit(x=xs_train, y=ys_train, 
                               validation_split=0.15, epochs=200, 
                               batch_size=200, verbose=1,
                               shuffle=True)

In [ ]:
hist3 = pd.DataFrame(training_history3.history)
hist3.tail()

In [ ]:
# Accuracy on training sets
accuracy_m3_train = model3.evaluate(xs_train, ys_train) # loss: 0.5169 - accuracy: 0.8095

print('Accuracy on train set', accuracy_m3_train)

In [ ]:
# Accuracy
plt.plot(hist3['accuracy'])
plt.plot(hist3['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
# Loss function
plt.plot(training_history3.history['loss'], label='Train')
plt.plot(training_history3.history['val_loss'], label='Validation')
plt.ylabel('Binary Cross Entropy Loss')
plt.xlabel('Epoch')
plt.title('Autoencoder Reconstruction Loss', pad=13)
plt.legend(loc='upper right')

Evaluate and predict on the test set

In [ ]:
model3_test_predict = model3.evaluate(xs_test, ys_test) # loss: 0.4480 - accuracy: 0.8546

In [ ]:
test_prediction3 = model3.predict(xs_test)
print("shape is {}".format(test_prediction3.shape))  
test_prediction3

#### Model 4


Define and compile the model

In [ ]:
# Parameters model 4
activation_fct = 'relu'
loss_function = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
optimizer = keras.optimizers.Adam(learning_rate=0.001)

# model 4
model4 = keras.Sequential([
    layers.Dense(3, kernel_regularizer=regularizers.l2(0.001), activation=activation_fct, input_shape=(3,)),
    layers.Dense(30, kernel_regularizer=regularizers.l2(0.001), activation=activation_fct),
    layers.Dense(16, kernel_regularizer=regularizers.l2(0.001), activation=activation_fct),
    layers.Dense(11)
  ])
model4.summary()

# compile the model 
model4.compile(optimizer=optimizer, loss=loss_function, metrics=['accuracy'])


In [ ]:
training_history4 = model4.fit(x=xs_train, y=ys_train, 
                               validation_split=0.15, epochs=200, 
                               batch_size=200, verbose=1,
                               shuffle=True)

In [ ]:
hist4 = pd.DataFrame(training_history4.history)
hist4.tail()

In [ ]:
# Accuracy on training sets
accuracy_m4_train = model4.evaluate(xs_train, ys_train) # loss: 0.4072 - accuracy: 0.8765

print('Accuracy on train set', accuracy_m4_train)

In [ ]:
plt.plot(hist4['accuracy'])
plt.plot(hist4['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
model4_test_predict = model4.evaluate(xs_test, ys_test) # loss: 0.4553 - accuracy: 0.8566

#### Model 5


Define and compile the model

In [ ]:
def create_model(activation='relu'):
  # create model
  model5 = keras.Sequential([
    layers.Dense(3, kernel_regularizer=regularizers.l2(0.001), activation='relu', input_shape=(3,)),
    layers.Dense(24, kernel_regularizer=regularizers.l2(0.001), activation='relu'),
    layers.Dense(24, kernel_regularizer=regularizers.l2(0.001), activation='relu'),
    layers.Dense(16, kernel_regularizer=regularizers.l2(0.001), activation='Softmax'),
    layers.Dense(11)
  ])
  # Compile model
  model5.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), 
                 optimizer=keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])
  return model5

In [ ]:
# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
activation_fct = ['softmax', 'relu']
#activation_fct = ['tanh', 'hard_sigmoid', 'linear', 'sigmoid']
param_grid = dict(optimizer__learning_rate=learn_rate, optimizer__momentum=momentum, model__activation=activation_fct)

# other parameters
loss_function = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
optimizer = keras.optimizers.Adam(learning_rate=0.001)

# model 5
# create model
model5 = KerasClassifier(model=create_model, epochs=150, batch_size=200, verbose=0)

grid_result5 = GridSearchCV(estimator=model5, param_grid=param_grid, n_jobs=-1, cv=5)

grid_result5.fit(xs_train, ys_train)


In [ ]:
# print best parameter after tuning
print(grid_result5.best_params_)
best_param5 = grid_result5.best_params_
  
# print how our model looks after hyper-parameter tuning
print(grid_result5.best_estimator_)

In [ ]:
# Accuracy
accuracy_model5_train = grid_result5.score(xs_train, ys_train)
model5_test_predict = grid_result5.score(xs_test, ys_test)

print('Accuracy of model (train)', accuracy_model5_train)
print('Accuracy of model (test)', model5_test_predict) 

#### Model 6

Based on the model 3 parameters, this section will be used to adjust the learning rate

In [ ]:
# Parameters model 6
activation_fct = 'relu'
loss_function = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
optimizer = keras.optimizers.Adam(learning_rate=0.0005)

# model 6
model6 = keras.Sequential([
    layers.Dense(3, kernel_regularizer=regularizers.l2(0.001), activation=activation_fct, input_shape=(3,)),
    layers.Dense(24, kernel_regularizer=regularizers.l2(0.001), activation=activation_fct),
    layers.Dense(24, kernel_regularizer=regularizers.l2(0.001), activation=activation_fct),
    layers.Dense(16, kernel_regularizer=regularizers.l2(0.001), activation=activation_fct),
    layers.Dense(11)
  ])
model6.summary()

# compile the model 6
model6.compile(optimizer=optimizer, loss=loss_function, metrics=['accuracy'])



In [ ]:
# Fit on training set
training_history6 = model6.fit(x=xs_train, y=ys_train, 
                               validation_split=0.15, epochs=275, 
                               batch_size=300, verbose=0,
                               shuffle=True)


In [ ]:
# Accuracy on training sets
accuracy_m6_train = model6.evaluate(xs_train, ys_train) # loss: 0.5171 - accuracy: 0.8288

print('Accuracy on train set', accuracy_m6_train)

In [ ]:
# Accuracy
plt.plot(training_history6.history['accuracy'], label='Train')
plt.plot(training_history6.history['val_accuracy'], label='Validation')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.title('Accuracy', pad=13)
plt.legend(loc='upper right')

In [ ]:
plt.plot(training_history6.history['loss'], label='Train')
plt.plot(training_history6.history['val_loss'], label='Validation')
plt.ylabel('Binary Cross Entropy Loss')
plt.xlabel('Epoch')
plt.title('Autoencoder Reconstruction Loss', pad=13)
plt.legend(loc='upper right')

In [ ]:
model6_test_predict = model6.evaluate(xs_test, ys_test) # loss: 0.5570 - accuracy: 0.8121

#### Model 7

In [ ]:
# Parameters model 7
activation_fct = 'relu'
loss_function = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
optimizer = keras.optimizers.Adam(learning_rate=0.0005)

# model 7
model7 = keras.Sequential([
    layers.Dense(3, kernel_regularizer=regularizers.l2(0.001), activation=activation_fct, input_shape=(3,)),
    layers.Dense(24, kernel_regularizer=regularizers.l2(0.001), activation=activation_fct),
    layers.Dense(24, kernel_regularizer=regularizers.l2(0.001), activation=activation_fct),
    layers.Dense(16, kernel_regularizer=regularizers.l2(0.001), activation=activation_fct),
    layers.Dense(11)
  ])
model7.summary()

# compile the model 6
model7.compile(optimizer=optimizer, loss=loss_function, metrics=['accuracy'])


In [ ]:
# Fit on training set
training_history7 = model7.fit(x=xs_train, y=ys_train, 
                               validation_split=0.15, epochs=300, 
                               batch_size=315, verbose=0,
                               shuffle=True)


In [ ]:
# Accuracy on training sets
accuracy_m7_train = model7.evaluate(xs_train, ys_train) # loss: loss: 0.5633 - accuracy: 0.8052
print('Accuracy on train set', accuracy_m7_train)

In [ ]:
plt.plot(training_history7.history['loss'], label='Train')
plt.plot(training_history7.history['val_loss'], label='Validation')
plt.ylabel('Binary Cross Entropy Loss')
plt.xlabel('Epoch')
plt.title('Autoencoder Reconstruction Loss', pad=13)
plt.legend(loc='upper right')

In [ ]:
model7_test_predict = model7.evaluate(xs_test, ys_test) # loss: 0.5914 - accuracy: 0.7943

## Compare Accuray of all the models

In [ ]:
df_accuracy = pd.DataFrame([model1_test_predict, model2_test_predict, model3_test_predict, model4_test_predict, [0, model5_test_predict], model6_test_predict, model7_test_predict])

In [ ]:
df_accuracy = df_accuracy.rename({0: 'loss', 1: 'accuracy'}, axis=1)
df_accuracy["model"] = ["model1", "model2", "model3", "model4", "model5", "model6", "model7"]

In [ ]:
df_accuracy.sort_values('accuracy') #  => best model is model 4

#### Evaluate performances

In [ ]:
model4_predictions = model4.predict(xs_test)
model4_predictions 

In [ ]:
# convert results into labelled colors
dict_colors = {}

for ind, val in enumerate(list(df_colors.columns.values)[3:]):
    dict_colors[ind] = val

print(dict_colors)

In [ ]:
def convert_tocolor(predictions):
    pred_max = np.argmax(predictions, axis=1)
    labelled_colors_pred = []
    for i in list(pred_max):
        labelled_colors_pred.append(dict_colors[i])
    return labelled_colors_pred

In [ ]:
colors_pred = convert_tocolor(model4_predictions)
colors_pred

In [ ]:
target_names = ['Black', 'Blue', 'Brown', 'Green', 'Grey', 'Orange', 'Pink', 'Purple', 'Red', 'White', 'Yellow']
model4_predictions = np.argmax(model4_predictions, axis=1)

# Confusion matrix
confusion_mat = confusion_matrix(ys_test_list, model4_predictions)
cm_display = ConfusionMatrixDisplay(confusion_matrix = confusion_mat, display_labels = target_names)
cm_display.plot()
plt.show()

# model performance report
print(classification_report(ys_test_list, model4_predictions, target_names=target_names))

Test with other data

In [ ]:
xs_test_other = np.array([[250/255, 104/255, 0], [170, 0, 255/255]])

convert_tocolor(model4.predict(xs_test_other))

## Towards Arduino

ref: https://colab.research.google.com/github/arduino/ArduinoTensorFlowLiteTutorials/blob/master/GestureToEmoji/arduino_tinyml_workshop.ipynb#scrollTo=0Xn1-Rn9Cp_8

https://colab.research.google.com/github/ucl-casa-ce/casa0018/blob/main/Week4/CASA0018_4_1_train_hello_world_model.ipynb#scrollTo=1muAoUm8lSXL

### Convert the Trained Model n° 4 to Tensor Flow Lite
In the below cell, we convert the model *4* format into TFlite format

In [ ]:
import os

MODELS_DIR = 'models/'
if not os.path.exists(MODELS_DIR):
    os.mkdir(MODELS_DIR)
MODEL_TF = MODELS_DIR + 'model'
MODEL_NO_QUANT_TFLITE = MODELS_DIR + 'model_no_quant.tflite'
MODEL_TFLITE = MODELS_DIR + 'model.tflite'
MODEL_TFLITE_MICRO = MODELS_DIR + 'model.cc'

In [ ]:
# Save the model to disk
model4.save(MODEL_TF)

In [ ]:
# Convert the model to the TensorFlow Lite format without quantization
converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_TF)
tflite_model = converter.convert()

# Save the model to disk
open(MODEL_NO_QUANT_TFLITE, "wb").write(tflite_model)

# Define a generator function that provides our test data's x values as a representative dataset, and tell the converter to use it
def representative_dataset_generator():
  for value in xs_train:
    # Each scalar value must be inside of a 2D array that is wrapped in a list
    yield [np.array(value/255, dtype=np.float32, ndmin=3)]

# Indicate that we want to perform the default optimizations, which include quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]

converter.representative_dataset = representative_dataset_generator

# Convert the model
tflite_model = converter.convert()

# Save the model to disk
open(MODEL_TFLITE, "wb").write(tflite_model)

In [ ]:


# Convert the model to the TensorFlow Lite format without quantization
converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_TF)
model_no_quant_tflite = converter.convert()

# Save the model to disk
open(MODEL_NO_QUANT_TFLITE, "wb").write(model_no_quant_tflite)

# Convert the model to the TensorFlow Lite format with quantization
def representative_dataset_generator():
  for value in xs_train:
    liste = value/255
    yield [np.array(liste, dtype=np.float32, ndmin=3)]

# Set the optimization flag - DEFAULT includes quantization.
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Enforce integer only quantization to reduce model size
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

# Provide a representative dataset to ensure we quantize correctly.
converter.representative_dataset = representative_dataset_generator

model_tflite = converter.convert()

# Save the model to disk
open(MODEL_TFLITE, "wb").write(model_tflite)

In [ ]:
# Convert the model to the TensorFlow Lite format without quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model4)
tflite_model = converter.convert()

# Save the model to disk
open("colourdetec_model.tflite", "wb").write(tflite_model)
  

basic_model_size = os.path.getsize("colourdetec_model.tflite")
print("Model is %d bytes" % basic_model_size)
  

### Convert the Trained Model n° 4 to be used in Arduino

In [ ]:
!echo "const unsigned char model[] = {" > /content/model_colour.h
!cat MODEL_TFLITE | xxd -i      >> /content/model_colour.h
!echo "};"                              >> /content/model_colour.h

import os
model_h_size = os.path.getsize("model_colour.h")
print(f"Header file, model_colour.h, is {model_h_size:,} bytes.")

#### Compare performance after conversion


In [ ]:
def predict_tflite(tflite_model, x_test):
  # Prepare the test data
  x_test_ = x_test.copy()
  #x_test_ = x_test_.reshape((x_test.size, 1))
  x_test_ = x_test_.astype(np.float32)

  # Initialize the TFLite interpreter
  interpreter = tf.lite.Interpreter(model_content=tflite_model)
  interpreter.allocate_tensors()

  input_details = interpreter.get_input_details()[0]
  output_details = interpreter.get_output_details()[0]

  # If required, quantize the input layer (from float to integer)
  input_scale, input_zero_point = input_details["quantization"]
  if (input_scale, input_zero_point) != (0.0, 0):
    x_test_ = x_test_ / input_scale + input_zero_point
    x_test_ = x_test_.astype(input_details["dtype"])
  
  # Invoke the interpreter
  y_pred = np.empty(x_test_.size, dtype=output_details["dtype"])
  for i in range(len(x_test_)):
    interpreter.set_tensor(input_details["index"], [x_test_[i]])
    interpreter.invoke()
    y_pred[i] = np.argmax(interpreter.get_tensor(output_details["index"])[[0]])
  
  # If required, dequantized the output layer (from integer to float)
  output_scale, output_zero_point = output_details["quantization"]
  if (output_scale, output_zero_point) != (0.0, 0):
    y_pred = y_pred.astype(np.float32)
    y_pred = (y_pred - output_zero_point) * output_scale

  return y_pred

def evaluate_tflite(tflite_model, x_test, y_true):
  global model4
  y_pred = predict_tflite(tflite_model, x_test)
  loss_function = tf.keras.losses.get(model4.loss)
  loss = loss_function(y_true, y_pred).numpy()
  return loss

In [ ]:
# Calculate predictions
y_test_pred_tf = model4.predict(xs_test)
y_test_pred_no_quant_tflite = predict_tflite(model_no_quant_tflite, xs_test)
y_test_pred_tflite = predict_tflite(model_tflite, xs_test)

### Generate a TensorFlow Lite for Microcontrollers Model

In [ ]:
# Install xxd if it is not available
!apt-get update && apt-get -qq install xxd
# Convert to a C source file, i.e, a TensorFlow Lite for Microcontrollers model
!xxd -i {MODEL_TFLITE} > {MODEL_TFLITE_MICRO}

In [ ]:
MODEL_TFLITE_MICRO

In [ ]:
# Print the C source file
!cat {MODEL_TFLITE_MICRO}
